<a href="https://colab.research.google.com/github/buganart/descriptor-transformer/blob/main/descriptor_model_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Check GPU, should be a Tesla V100
!nvidia-smi -L
import os
print(f"We have {os.cpu_count()} CPU cores.")

In [ ]:
#@markdown Mount google drive
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    raise RuntimeError(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        output.clear()
        print(msg)

In [ ]:
#@markdown Install wandb and log in
%pip install wandb
output.clear()
import wandb
from pathlib import Path
wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login
output.clear()
print("ok!")

In [ ]:
#@title Configuration

#@markdown Directories can be found via file explorer on the left by navigating into `drive` to the desired folders. 
#@markdown Then right-click and `Copy path`.
test_data_path = "/content/drive/My Drive/AUDIO DATABASE/MUSIC TRANSFORMER/Transformer Corpus/" #@param {type:"string"}

#@markdown ### Resumption of previous runs
#@markdown Optional resumption arguments below, leaving both empty will start a new run from scratch. 
#@markdown - The ID can be found on wandb. 
#@markdown - It's 8 characters long and may contain a-z letters and digits (for example `1t212ycn`).

#@markdown Resume a previous run 
resume_run_id = "2dq79lcy" #@param {type:"string"}

#@markdown the number of predicted descriptors after the test_data
prediction_length = 10 #@param {type:"integer"}

#@markdown the path to save all generated descriptors as json
prediction_output_dir = "/content/drive/My Drive/Descriptor Model/OUTPUTS/" #@param {type:"string"}

import re
from pathlib import Path
from argparse import Namespace

def check_wandb_id(run_id):
    if run_id and not re.match(r"^[\da-z]{8}$", run_id):
        raise RuntimeError(
            "Run ID needs to be 8 characters long and contain only letters a-z and digits.\n"
            f"Got \"{run_id}\""
        )

check_wandb_id(resume_run_id)

prediction_output_dir = Path(prediction_output_dir)
prediction_output_dir.mkdir(parents=True, exist_ok=True)


colab_config = {
    "resume_run_id": resume_run_id,
    "test_data_path": test_data_path,
    "prediction_length": prediction_length,
    "prediction_output_dir": prediction_output_dir,
}

for k, v in colab_config.items():
    print(f"=> {k:20}: {v}")

config = Namespace(**colab_config)
config.seed = 1234

In [ ]:
#@markdown Install dependency
%pip install --upgrade git+https://github.com/buganart/descriptor-transformer.git#egg=desc
from desc.train_function import get_resume_run_config, init_wandb_run, setup_model, setup_datamodule
from desc.helper_function import save_descriptor_as_json
clear_on_success()

# generate

In [ ]:
config = get_resume_run_config(resume_run_id)
config.resume_run_id = resume_run_id
run = init_wandb_run(config, run_dir="./", mode="offline")
model,_ = setup_model(config, run)
model.eval()
#construct test_data
testdatamodule = setup_datamodule(config, run, isTrain=False)
test_dataloader = testdatamodule.test_dataloader()
test_data, fileindex = next(iter(test_dataloader))

prediction = model.predict(test_data, prediction_length)

#un normalize output
prediction = (prediction + testdatamodule.dataset_mean) * testdatamodule.dataset_std

save_descriptor_as_json(prediction_output_dir, prediction, fileindex, testdatamodule, resume_run_id)
print("ok!")